# TẠO CÁC TẬP DỮ LIỆU TRAIN, TEST (SPLITS)

## 1. Yêu cầu chung
Tạo ra các splits, mỗi split tương ứng với một tập dữ liệu train - test.

### Bài học lý thuyết
- [Tutorial on k-fold cross-validation](https://www.kaggle.com/code/satishgunjal/tutorial-k-fold-cross-validation)
- [Training, Validation, Test Split and Cross Validation Done Right](https://machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/)
- [How to Configure k-Fold Cross Validation](https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/)

![KFold Cross Validation](https://raw.githubusercontent.com/satishgunjal/images/master/KFold_Cross_Validation.png)

## 2. Yêu cầu cụ thể
- **Input:**
  + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Yamaha, Others). Ví dụ: [Google Drive Link](https://drive.google.com/drive/u/1/folders/12WrC9APRzQX36cataTccBiN_Z5F2C0yD)
  + Các ảnh được đặt tên theo quy ước: [Naming Convention](https://colab.research.google.com/drive/1sZBm78OiTUOqbg5-Z2UOKlk3B37TAgd8)
  + Số splits NumSplits - mặc định NumSplits=5 (tương đương 5-fold CV)

- **Output:**
  + File `MotocycleDataset.csv` - Tập tin chứa tất cả ảnh của dataset.
    - Chương trình sẽ scan qua cây thư mục để tìm tất cả các ảnh (chỉ chọn các ảnh có định dạng và phần mở rộng là `.jpg`).
    - Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy:
      - `ImageFullPath` ở dạng `<Thư mục Hiệu xe>/<Tên ảnh>`. Ví dụ: `Honda/2024123.Honda.1.jpg`.
      - `CategoryID` là số nguyên thuộc [0..5] theo quy ước:
        - 0: Others
        - 1: Honda
        - 2: Suzuki
        - 3: Yamaha
        - 4: VinFast

  + File `MotocycleDataset-Splits-[1..5]-[Train/Test].csv` - Phân chia thành các splits, mỗi split gồm các ảnh được chia thành thành 2 tập Train - Test.
    - Chương trình sẽ đọc dữ liệu từ file `MotocycleDataset.csv`, sau đó với mỗi hiệu xe, chia ngẫu nhiên thành 5 tập dữ liệu để đảm bảo mỗi tập dữ liệu Train/Test có dữ liệu của các hiệu xe.
    - Từ 5 tập dữ liệu này, tạo ra 5 tập dữ liệu lớn hơn sao cho mỗi tập dữ liệu này chứa đủ dữ liệu của tất cả các hiệu xe.
    - Tạo ra 5 splits từ các tập dữ liệu này và ghi xuống thành 2 tập tin tương ứng với Train và Test.

- **Lưu ý:**
  - Cần viết thêm các cell để:
    - Hiển thị danh sách các tên tập tin ảnh trong từng Split-Train/Test.
    - Thống kê số lượng ảnh cho từng CategoryID trong mỗi Split-Train/Test.
  - Cần có chú thích và giải thích logic của từng bước.

## 3. Nộp bài
Sinh viên nộp notebook. Các bài nộp sớm sẽ được full điểm. Deadline: 17:00 - 10/06/2024.

## 4. Bài làm đạt yêu cầu
Sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.

## Thông tin của tác giả, ngày cập nhật

SyntaxError: invalid syntax (1499702342.py, line 2)

# **Thành Viên**
- Võ Minh Tùng - 22521623
- Trần Xuân Ánh

In [18]:
import os
import csv
import cv2
from sklearn.model_selection import KFold
import pandas as pd
import matplotlib.pyplot as plt

# Define dataset path
dataset_path = 'data'

# Define the class names
class_names = {
    'Others': 0,
    'Honda': 1,
    'Suzuki': 2,
    'Yamaha': 3,
    'VinFast': 4
}

# Constants
NumImgsPerRow = 10
ImgHeight = ImgWidth = 150
figsize_per_image = (2, 2)
classes = 5  # 5 categories -> 5 classes
categories = ['Others', 'Honda', 'Suzuki', 'Yamaha', 'VinFast']
category_mapping = {category: idx for idx, category in enumerate(categories)}
dataset_path = './data'

In [19]:
# Initialize lists to hold image data and labels
X = []
y = []

# Scan through the dataset directory structure
for class_name, class_id in class_names.items():
    class_path = os.path.join(dataset_path, class_name)
    print(class_path)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        #print(img_path.split('.')[-1].lower()) 
        if img_path.split('.')[-1].lower() == 'jpg':  # Only consider .jpg files
            X.append(f'{class_name}/{img_name}')
            y.append(class_id)

# Save the dataset to MotocycleDataset.csv
with open('MotocycleDataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ImageFullPath', 'CategoryID'])
    for img_path, class_id in zip(X, y):
        writer.writerow([img_path, class_id])


./data/Others


FileNotFoundError: [Errno 2] No such file or directory: './data/Others'

In [ ]:
# Define KFold cross-validator with 5 splits, shuffling data, and fixed random state
kf = KFold(n_splits=5, shuffle=True, random_state=1337)

# Perform the 5-fold split and save to CSV files
for i, (train_index, test_index) in enumerate(kf.split(X)):
    train_data = [(X[idx], y[idx]) for idx in train_index]
    test_data = [(X[idx], y[idx]) for idx in test_index]

    # Save training data
    with open(f'MotocycleDataset-Splits-{i+1}-Train.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ImageFullPath', 'CategoryID'])
        writer.writerows(train_data)
    
    # Save testing data
    with open(f'MotocycleDataset-Splits-{i+1}-Test.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ImageFullPath', 'CategoryID'])
        writer.writerows(test_data)
    

In [ ]:
# Function to display images
def display_images(data, title):
    images = {}
    for Class in range(classes):
        class_df = data[data['CategoryID'] == Class]
        sample_images = class_df.sample(n=min(NumImgsPerRow, len(class_df)))['ImageFullPath']
        images[Class] = [cv2.resize(cv2.imread(os.path.join(dataset_path, img_path)), (ImgWidth, ImgHeight)) for img_path in sample_images]

    figsize = (figsize_per_image[0] * (NumImgsPerRow + 1), figsize_per_image[1] * classes)
    fig, axes = plt.subplots(classes, NumImgsPerRow + 1, figsize=figsize)

    for i in range(classes):
        for j in range(NumImgsPerRow + 1):
            ax = axes[i, j]
            if j == 0:
                ax.text(0.5, 0.5, f'{categories[i]}', va='center', ha='center', transform=ax.transAxes, fontsize=14, fontweight='bold')
            elif j-1 < len(images[i]):
                ax.imshow(cv2.cvtColor(images[i][j-1], cv2.COLOR_BGR2RGB))
            ax.axis('off')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


# Perform the 5-fold split and save to CSV files
for i, (train_index, test_index) in enumerate(kf.split(X)):
    train_data = [(X[idx], y[idx]) for idx in train_index]
    test_data = [(X[idx], y[idx]) for idx in test_index]
    
    train_df = pd.DataFrame(train_data, columns=['ImageFullPath', 'CategoryID'])
    test_df = pd.DataFrame(test_data, columns=['ImageFullPath', 'CategoryID'])
    
    # Save training data
    train_filename = f'MotocycleDataset-Splits-{i+1}-Train.csv'
    train_df.to_csv(train_filename, index=False)
    
    # Save testing data
    test_filename = f'MotocycleDataset-Splits-{i+1}-Test.csv'
    test_df.to_csv(test_filename, index=False)

    # Display filenames and category statistics for each split
    print(f"Split-{i+1} Train:")
    print(train_df['ImageFullPath'].to_list())
    print(f"Split-{i+1} Test:")
    print(test_df['ImageFullPath'].to_list())

    print(f"Split-{i+1} Train Category Distribution:")
    print(train_df['CategoryID'].value_counts())
    print(f"Split-{i+1} Test Category Distribution:")
    print(test_df['CategoryID'].value_counts())

    # Display images in each Split-Train/Test
    display_images(train_df, f'Split-{i+1} Train')
    display_images(test_df, f'Split-{i+1} Test')

    # Visualize category distribution
    plt.figure(figsize=(12, 6))
    
    # Train category distribution
    plt.subplot(1, 2, 1)
    train_counts = train_df['CategoryID'].value_counts().sort_index()
    train_counts.plot(kind='bar')
    plt.title(f"Split-{i+1} Train Category Distribution")
    plt.xlabel("CategoryID")
    plt.ylabel("Number of Images")
    plt.xticks(rotation=0)

    # Test category distribution
    plt.subplot(1, 2, 2)
    test_counts = test_df['CategoryID'].value_counts().sort_index()
    test_counts.plot(kind='bar')
    plt.title(f"Split-{i+1} Test Category Distribution")
    plt.xlabel("CategoryID")
    plt.ylabel("Number of Images")
    plt.xticks(rotation=0)

    plt.tight_layout()
    plt.show()

    print("======================================\n")